In [112]:
import pandas as pd
from Bio import PDB 
from Bio.PDB import PDBParser
from simba2 import methods as simba

In [113]:
# Import training and test datasets
b1131 = pd.read_csv('../data/processed/B1131_expddg.csv')
b663 =  pd.read_csv('../data/processed/B663_expddg.csv')
s350 =   pd.read_csv('../data/processed/S350_expddg.csv')

# Convert No column to string for compatibility with simba output
b1131.No = b1131.No.astype('str')
b663.No = b663.No.astype('str')
s350.No = s350.No.astype('str')

In [114]:
pdb_dir = '../data/external/PDB'

In [115]:
# Dataframe containing unique entries across datasets
dataset_residues = (pd.concat([b1131, b663, s350])
 .drop(columns = ['exp_ddG'])
 .drop_duplicates())

# List of unique PDBs across datasets
dataset_pdbs = dataset_residues['PDB'].unique()

In [123]:
# Calculate RSA, Hdiff, Vdiff, and predicted ddG (predicted ddG is not used for creating datasets) with Simba2 
# for each PDB in datasets and indicate if those entries are present in the datasets.
# Insert column indicating if PDB is a heterooligomer
df_list = []
for pdb in dataset_pdbs:
    pdb_path, exists = simba.exists_pdb(pdb, pdb_dir)
    print(pdb_path, exists)
    if exists:
        df = simba.simba2_predict(pdb, pdb_path)
        multichain, homo = simba.check_chains(df)
        df.insert(loc=1, column='Hetero', value=multichain and not homo)
        df_list.append(pd.merge(df, 
                 dataset_residues[dataset_residues['PDB'] == pdb],
                 how = 'outer',
                 indicator = True,       
                 left_on = ['PDB', 'Number', 'Wild', 'Mutated'],
                 right_on = ['PDB', 'No', 'Wild', 'Mutated']))

../data/external/PDB\pdb1a2j.ent True
../data/external/PDB\pdb1a2p.ent True
../data/external/PDB\pdb1a6m.ent True
../data/external/PDB\pdb1aar.ent True
../data/external/PDB\pdb1aep.ent True
../data/external/PDB\pdb1aps.ent True
../data/external/PDB\pdb1arr.ent True
../data/external/PDB\pdb1ay7.ent True


KeyboardInterrupt: 

In [88]:
simba_df = pd.concat(df_list)        

In [89]:
def len_unique(inputlist):
    return len(inputlist.unique())

In [90]:
# Check which residues in the datasets that were not merged with Simba output

absent = simba_df[simba_df['_merge'] == 'right_only']
print('B1131:', pd.merge(b1131, absent, on = ['PDB', 'Wild', 'No', 'Mutated'], how = 'inner'))
print('B663:', pd.merge(b663, absent, on = ['PDB', 'Wild', 'No', 'Mutated'], how = 'inner'))
print('S350:', pd.merge(s350, absent, on = ['PDB', 'Wild', 'No', 'Mutated'], how = 'inner'))

B1131: Empty DataFrame
Columns: [PDB, Wild, No, Mutated, exp_ddG, Hetero, Gene, Chain, Number, RSA, Hdiff, Vdiff, ddG_SimBa-IB, ddG_SimBa-SYM, _merge, RSA_mean, ddG_SimBa-IB_mean, ddG_SimBa-SYM_mean]
Index: []
B663:     PDB Wild   No Mutated  exp_ddG Hetero Gene Chain Number  RSA  Hdiff  \
0  1YCC    N   57       I     4.20    NaN  NaN   NaN    NaN  NaN    NaN   
1  1FGA    C   83       S    -1.91    NaN  NaN   NaN    NaN  NaN    NaN   
2  1FGA    C  117       S    -0.62    NaN  NaN   NaN    NaN  NaN    NaN   
3  1FGA    H   93       G     1.60    NaN  NaN   NaN    NaN  NaN    NaN   

   Vdiff  ddG_SimBa-IB  ddG_SimBa-SYM      _merge  RSA_mean  \
0    NaN           NaN            NaN  right_only       NaN   
1    NaN           NaN            NaN  right_only       NaN   
2    NaN           NaN            NaN  right_only       NaN   
3    NaN           NaN            NaN  right_only       NaN   

   ddG_SimBa-IB_mean  ddG_SimBa-SYM_mean  
0                NaN                 NaN  
1     

Four data points in B663 does not exist in the simba output (the residue type is not at that position in the PDBs)

In [91]:
# remove outliers
b1131 = b1131[(b1131['exp_ddG'] > -8) & (b1131['exp_ddG'] < 8)]
b663 = b663[(b663['exp_ddG'] > -8) & (b663['exp_ddG'] < 8)]
s350 = s350[(s350['exp_ddG'] > -8) & (s350['exp_ddG'] < 8)]

In [92]:
len(b1131)

1118

In [93]:
len(b663)

661

In [94]:
len(s350)

350

In [95]:
# Create dataframe with those entries in the dataset where it is not
# possible to know which chain it is located in. 

## keep only mutations that are present in the data sets
simba_df = simba_df[simba_df['_merge'] == 'both'].drop(columns = ['_merge'])

## keep heterooligomers
hetero = simba_df[simba_df['Hetero']] 
hetero[['PDB','Number', 'Mutated']].drop_duplicates() 

## for each unique mutation, count chains
no_chains = pd.DataFrame(hetero.groupby(['PDB', 'Number', 'Mutated'])['Chain'].agg(len_unique))

## make dataframe of mutations with more than one chain
unsure_chain = no_chains[no_chains['Chain'] != 1]
unsure_chain = pd.DataFrame(list(unsure_chain.index), columns = ['PDB', 'No', 'Mutated'])

unsure_chain

,PDB,No,Mutated
0,1AON,48,W
1,1JIW,10,A
2,1OTR,34,C
3,1OTR,34,W
4,1WAA,9,F
5,1WAA,9,P
6,1WAA,9,S
7,1ZNJ,10,E
8,1ZNJ,10,T
9,1ZNJ,25,D


In [96]:
hetero

,PDB,Hetero,Gene,Chain,Number,Wild,RSA,Mutated,Hdiff,Vdiff,ddG_SimBa-IB,ddG_SimBa-SYM,No,RSA_mean,ddG_SimBa-IB_mean,ddG_SimBa-SYM_mean
2720,1AY7,True,,B,40,C,0.020509,A,2.6,-0.199,-1.865244,-1.375502,40,NaN,NaN,NaN
3409,1AY7,True,,B,75,R,0.411749,L,-5.6,-0.067,0.229753,0.605316,75,NaN,NaN,NaN
3413,1AY7,True,,B,75,R,0.411749,Q,0.0,-0.296,-0.558877,-0.252688,75,NaN,NaN,NaN
3473,1AY7,True,,B,78,K,0.519951,Q,-0.8,-0.248,-0.350227,-0.131050,78,NaN,NaN,NaN
1084,1WAA,True,,A,9,Y,0.796644,F,-1.0,-0.037,-0.112586,-0.171528,9,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5413,1ZNJ,True,,D,25,F,0.377160,D,7.8,-0.788,-2.038073,-1.845988,25,NaN,NaN,NaN
5414,1ZNJ,True,,F,25,F,0.140711,D,7.8,-0.788,-3.681390,-3.558453,25,NaN,NaN,NaN
5415,1ZNJ,True,,H,25,F,0.463864,D,7.8,-0.788,-1.435484,-1.218043,25,NaN,NaN,NaN
5416,1ZNJ,True,,J,25,F,0.305432,D,7.8,-0.788,-2.536584,-2.365475,25,NaN,NaN,NaN


In [97]:
no_chains

Chain
PDB  Number Mutated       
1AON 48     W            7
1AY7 40     A            1
     75     L            1
            Q            1
     78     Q            1
1CSE 54     A            1
1JIW 10     A            2
     15     F            1
1K9Q 30     Y            1
1OTR 17     N            1
            T            1
     34     C            2
            W            2
     36     F            1
     41     L            1
            N            1
            S            1
            T            1
            V            1
     67     N            1
            S            1
            T            1
1TTQ 22     L            1
            V            1
1WAA 9      F            6
            P            6
            S            6
1ZNJ 10     E            6
            T            6
     25     D            6
3I40 10     D            1
            E            1
            T            1
     25     D            1
            H            1
     26     T            1
     8      H            1
     9      D            2
5CGQ 175    Q            1
     22     I            1
            V            1

In [98]:
def anti_join(dataset, unsure):
    df = pd.merge(dataset, unsure, on = ['PDB', 'No', 'Mutated'], how = 'outer', indicator = True)
    return df[df['_merge'] == 'left_only'].drop(columns = ['_merge'])

In [99]:
# Discard entries where it's not possible to know the chain
b1131 = anti_join(b1131, unsure_chain)
b663 = anti_join(b663, unsure_chain)
s350 = anti_join(s350, unsure_chain)

In [100]:
len(b1131)

1114

In [101]:
len(b663)

659

In [102]:
len(s350)

345

In [103]:
simba_df

,PDB,Hetero,Gene,Chain,Number,Wild,RSA,Mutated,Hdiff,Vdiff,ddG_SimBa-IB,ddG_SimBa-SYM,No,RSA_mean,ddG_SimBa-IB_mean,ddG_SimBa-SYM_mean
655,1A2J,False,,A,33,C,0.003240,S,3.7,-0.195,-2.307626,-1.871341,33,NaN,NaN,NaN
216,1A2P,False,,A,6,T,0.498314,P,-0.6,-0.034,-0.231597,0.009016,6,0.505305,-0.227546,0.006355
217,1A2P,False,,B,6,T,0.503293,P,-0.6,-0.034,-0.228711,0.007121,6,0.505305,-0.227546,0.006355
218,1A2P,False,,C,6,T,0.514306,P,-0.6,-0.034,-0.222329,0.002930,6,0.505305,-0.227546,0.006355
267,1A2P,False,,A,7,F,0.165842,L,-0.3,-0.232,-0.735636,-0.214426,7,0.160114,-0.741715,-0.215634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2306,5DFR,False,,A,121,G,0.465551,H,1.4,0.931,0.306954,0.591416,121,NaN,NaN,NaN
2996,5DFR,False,,A,155,I,0.152181,T,3.4,-0.506,-2.076654,-1.732356,155,NaN,NaN,NaN
316,5PTI,False,,A,16,A,0.435029,T,0.5,0.275,-0.130736,0.172276,16,NaN,NaN,NaN
317,5PTI,False,,A,16,A,0.435029,V,-2.2,0.514,0.306246,0.638833,16,NaN,NaN,NaN


In [104]:
# Merge datasets (without unsure chains) with simba output and tidy up result 
def choose_var(row, variable):
    if pd.isna(row[variable + '_mean']):
        return row[variable]
    else:
        return row[variable + '_mean']

def finalize_dataset(dataset, simba_output):
    df = pd.merge(dataset, simba_output, on = ['PDB', 'No', 'Wild', 'Mutated'], how = 'inner')
    df['final_RSA'] = df.apply(lambda row : choose_var(row, 'RSA'), axis=1)
    df = df[['PDB', 'Wild', 'Number', 'final_RSA', 'Mutated', 'exp_ddG', 'Hdiff', 'Vdiff']]
    df = df.drop_duplicates()
    df = df.rename(columns = {"final_RSA" : "RSA"})
    df = df[['PDB', 'Wild', 'Number', 'RSA', 'Mutated', 'Hdiff', 'Vdiff', 'exp_ddG']]
    
    return df

In [105]:
b1131_simba2 = finalize_dataset(b1131, simba_df)
#b1131_simba2.to_csv('../data/b1131_simba2.csv', index = False)

b663_simba2 = finalize_dataset(b663, simba_df)
#b663_simba2.to_csv('../data/b663_simba2.csv', index = False)

s350_simba2 = finalize_dataset(s350, simba_df)
#s350_simba2.to_csv('../data/s350_simba2.csv', index = False)

In [106]:
print("After removal of hetero:", len(b1131))
print("Final:", len(b1131_simba2))
print("Duplicates:", sum(b1131.duplicated()))

After removal of hetero: 1114
Final: 1112
Duplicates: 2


There are two duplicated data points in the original B1131 which are removed above

In [107]:
## The duplicated entries are:
b1131[b1131.duplicated()]

,PDB,Wild,No,Mutated,exp_ddG
163,1BVC,A,144,L,0.4
191,1DPM,H,257,L,-2.5


In [108]:
print("After removal of hetero:", len(b663))
print("Final:", len(b663_simba2))
print("Duplicates:", sum(b663.duplicated()))

After removal of hetero: 659
Final: 655
Duplicates: 0


There were four missing data points in the PDBs, see above

In [109]:
print("After removal of hetero:", len(s350))
print("Final:", len(s350_simba2))
print("Duplicates:", sum(s350.duplicated()))

After removal of hetero: 345
Final: 344
Duplicates: 0


One missing data point due to obsolete PDB 2A01

In [110]:
# Save datasets as new datasets with new names

b1131_simba2['dataset'] = "B1112"
b663_simba2['dataset'] = "B655"
s350_simba2['dataset'] = "S344"

b1131_simba2.to_csv('../data/processed/B1112.csv', index = False)
b663_simba2.to_csv('../data/processed/B655.csv', index = False)
s350_simba2.to_csv('../data/processed/S344.csv', index = False)

In [121]:
from simba2 import methods as simba

In [117]:
dir(methods)

['HVdiff_table',
 'PDBParser',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'calc_RSA',
 'calc_simba_IB',
 'calc_simba_SYM',
 'check_chains',
 'data_file',
 'exists_pdb',
 'fs',
 'get_gene',
 'get_residueAreas',
 'join_hvdiff',
 'json',
 'mean_RSA',
 'naccess_config',
 'natsort_keygen',
 'os',
 'pd',
 'pkg_resources',
 'simba2_predict']